In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ModuleNotFoundError: ignored

In [3]:
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,GlobalMaxPool1D, Bidirectional 
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
from numpy import random
from random import choices
import os
from sklearn.utils import shuffle
from sklearn import metrics
import pickle
from scipy.io.wavfile import read,write
import pickle

from datetime import datetime
from packaging import version

tf.keras.backend.set_floatx('float32')

print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.4.1


In [4]:
os.chdir('/content/drive/My Drive/')
os.listdir()

FileNotFoundError: ignored

# To train on new data and dumping into a pickle file

In [ ]:
# KEYWORD_FOLDER1 = 'Data/Help/trimmed_audio'
# KEYWORD_FOLDER2 = 'Data/Help/Help_Data_10dB'
# KEYWORD_FOLDER3= 'Data/Help/Help_Data_Natural_10dB'

KEYWORD_FOLDER1 = 'Data/Help/Help_Data_Arijit1'
KEYWORD_FOLDER2 = 'Data/Help/Help_Data_Party1'
KEYWORD_FOLDER3= 'Data/Help/Help_Data_Market1'
KEYWORD_FOLDER4 = 'Data/Help/Help_Data_Lecture1'
KEYWORD_FOLDER5 = 'Data/Help/Help_Data_Impala1'
KEYWORD_FOLDER6= 'Data/Help/Help_Data_Choir1'
KEYWORD_FOLDER7 = 'Data/Help/Help_Data_Cheerful1'
KEYWORD_FOLDER8= 'Data/Help/Help_Data_Chatter1'

# KEYWORD_FOLDER4= 'Data/Help/Google_noise_mix_pink'
# KEYWORD_FOLDER5= 'Data/Help/Google_noise_mix_tap'
# KEYWORD_FOLDER6 = 'Data/Help/Google_noise_mix_dishes'
# KEYWORD_FOLDER7 = 'Data/Help/Google_noise_mix_bike'


NEGATIVE_FOLDER1 = 'Data/Negative/Negative_Data/'
NEGATIVE_FOLDER2 = 'Data/Negative/Negative_Data_10dB'
NEGATIVE_FOLDER3 = 'Data/Negative/Negative_Data_Natural_10dB'

KEYWORD_FOLDER_TEST = 'Data/Help/Crowd-Sourced'
# KEYWORD_FOLDER_TEST = 'Data/Help/Help_Data_Test_Old/'
# NEGATIVE_FOLDER_TEST = 'Data/Negative/Negative_Data_Test_Old/'

INPUT_SHAPE = (126,40)

In [ ]:
def count_files(folder, extension):
	count = 0
	for file in os.listdir(folder):
		if file.endswith(extension):
			file_path = os.path.join(folder, file)
			count += 1
	return count

In [ ]:
def load_data_folder(folder, is_keyword):
  num_samples = count_files(folder, '.wav')
  data_X = np.zeros((num_samples, INPUT_SHAPE[0], INPUT_SHAPE[1]), dtype=np.float64)
  data_Y = np.zeros((num_samples), dtype=np.float64)

  count = 0
  for file in os.listdir(folder):
    if file.endswith('.wav'):
      file_path = os.path.join(folder, file)
      y, sr = librosa.load(file_path,sr=None)
      mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=128, n_fft=256, n_mfcc=20)
      mfcc_delta = librosa.feature.delta(mfcc)[:10, :]
      mfcc_double_delta = librosa.feature.delta(mfcc, order=2)[:10, :]
      data_X[count, :, :20] = mfcc.T
      data_X[count, :, 20:30] = mfcc_delta.T
      data_X[count, :, 30:] = mfcc_double_delta.T
      data_Y[count] = int(is_keyword)
      count += 1
      if count%50==0:
        print(count)
  return data_X, data_Y

In [ ]:
def load_data(folders):
	num_samples = sum([count_files(folder, '.wav') for folder, is_keyword in folders])
	data_X = np.zeros((num_samples, INPUT_SHAPE[0], INPUT_SHAPE[1]), dtype=np.float64)
	data_Y = np.zeros((num_samples), dtype=np.float64)
	count = 0
	try:
		for folder, is_keyword in folders:
			num_samples_folder = count_files(folder, '.wav')
			data_X[count:count+num_samples_folder, :, :], data_Y[count:count+num_samples_folder] = (
				load_data_folder(folder, is_keyword))
			count += num_samples_folder
		return shuffle(data_X, data_Y, random_state=0)
	except:
		print('error')


In [ ]:
def load_train_data():
  #folders = [(NEGATIVE_FOLDER_TRAIN_1, False), (NEGATIVE_FOLDER_TRAIN_2, False), (NEGATIVE_FOLDER_TRAIN_3, False)]
  folders = [(KEYWORD_FOLDER1, True), (KEYWORD_FOLDER2, True), (KEYWORD_FOLDER3, True), (KEYWORD_FOLDER4, True), (KEYWORD_FOLDER5, True), (KEYWORD_FOLDER6, True), (KEYWORD_FOLDER7, True), (KEYWORD_FOLDER8, True)]
  # folders = [(KEYWORD_FOLDER4, True), (KEYWORD_FOLDER5, True), (KEYWORD_FOLDER6, True), (NEGATIVE_FOLDER1, False), (NEGATIVE_FOLDER2, False), (NEGATIVE_FOLDER3, False)]
  return load_data(folders)

In [ ]:
def load_test_data():
  #folders = [(NEGATIVE_FOLDER_TEST, True)]
  folders = [(KEYWORD_FOLDER_TEST, True)] 
  return load_data(folders)

In [ ]:
#Generate the MFCC matrix
# train_X0, train_Y0 = load_train_data()
# print("Train data extracted")

test_X, test_Y = load_test_data()
print("Test data extracted")

/usr/local/lib/python3.6/dist-packages/librosa/filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "


50
100
Test data extracted


In [ ]:
# Dump into a pickle
import pickle

with open("new_mix_help_train_x.pickle", "wb") as f:
  pickle.dump(test_X, f)
print("Train set features done")
with open("new_mix_help_train_y.pickle", "wb") as f:
  pickle.dump(test_Y, f)
print("Train set labels done")


# with open("help_trimmed_test_x.pickle", "wb") as f:
#   pickle.dump(test_X, f)
# print("Test set features done")
# with open("help_trimmed_test_y.pickle", "wb") as f:
#   pickle.dump(test_Y, f)
# print("Test set labels done")

Train set features done
Train set labels done


#To load data from an existing pickle

In [ ]:
os.chdir("Pickles")
os.listdir()

['Help',
 'Bachao',
 'Negative',
 'noise_bachao_train_x.pickle',
 'noise_bachao_train_y.pickle',
 'total_bachao_x_new.pickle',
 'total_bachao_y_new.pickle',
 'total_train_x_new.pickle',
 'total_train_y_new.pickle',
 'logs']

In [ ]:
# Load from a pickle
with open('Help/crowdsourced_train_x.pickle', 'rb') as f:
  train_X0cs = pickle.load(f)
print("Crowdsourced Train set features loaded")
with open('Help/crowdsourced_train_y.pickle', 'rb') as f:
  train_Y0cs = pickle.load(f)
print("Crowdsourced Train set labels loaded")

with open('Help/new_mix_help_train_x.pickle', 'rb') as f:
  train_X0h1 = pickle.load(f)
print("New Mix Help Train set features loaded")
with open('Help/new_mix_help_train_y.pickle', 'rb') as f:
  train_Y0h1 = pickle.load(f)
print("New Mix Help Train set labels loaded")

with open('Negative/youtube_train_x(1).pickle', 'rb') as f:
  train_X0y1 = pickle.load(f)
print("YouTube(1) Train set features loaded")
with open('Negative/youtube_train_y(1).pickle', 'rb') as f:
  train_Y0y1 = pickle.load(f)
print("YouTube(1) Train set labels loaded")

with open('Help/google_mix_help_train_x.pickle', 'rb') as f:
  train_X0hg = pickle.load(f)
print("Google mixed Help Train set features loaded")
with open('Help/google_mix_help_train_y.pickle', 'rb') as f:
  train_Y0hg = pickle.load(f)
print("Google mixed Help Train set labels loaded")

with open('Negative/google_trimmed_train_x.pickle', 'rb') as f:
  train_X0g = pickle.load(f)
print("Google Train set features loaded")
with open('Negative/google_trimmed_train_y.pickle', 'rb') as f:
  train_Y0g = pickle.load(f)
print("Google Train set labels loaded")

with open('Help/help_trimmed_train_x.pickle', 'rb') as f:
  train_X0h = pickle.load(f)
print("Train set features loaded")
with open('Help/help_trimmed_train_y.pickle', 'rb') as f:
  train_Y0h = pickle.load(f)
print("Train set labels loaded")

with open('Help/help_trimmed_test_x.pickle', 'rb') as f:
  test_X = pickle.load(f)
print("Test set features loaded")
with open('Help/help_trimmed_test_y.pickle', 'rb') as f:
  test_Y = pickle.load(f)
print("Test set labels loaded")

Crowdsourced Train set features loaded
Crowdsourced Train set labels loaded
New Mix Help Train set features loaded
New Mix Help Train set labels loaded
YouTube(1) Train set features loaded
YouTube(1) Train set labels loaded
Google mixed Help Train set features loaded
Google mixed Help Train set labels loaded
Google Train set features loaded
Google Train set labels loaded
Train set features loaded
Train set labels loaded
Test set features loaded
Test set labels loaded


In [ ]:
print(train_X0cs.shape, train_Y0cs.shape) #all true
print(train_X0h1.shape, train_Y0h1.shape) #all true
print(train_X0y1.shape, train_Y0y1.shape) #all false
print(train_X0hg.shape, train_Y0hg.shape) #all true
print(train_X0g.shape, train_Y0g.shape)   #all false
print(train_X0h.shape, train_Y0h.shape)   #2410/3606 trues
print(test_X.shape, test_Y.shape)         #100/139 trues

(134, 126, 40) (134,)
(6416, 126, 40) (6416,)
(19532, 126, 40) (19532,)
(4010, 126, 40) (4010,)
(24702, 126, 40) (24702,)
(3606, 126, 40) (3606,)
(139, 126, 40) (139,)


In [ ]:
print(train_Y0cs.sum())
print(train_Y0h1.sum())
print(train_Y0y1.sum())
print(train_Y0hg.sum())
print(train_Y0g.sum())
print(train_Y0h.sum())
print(test_Y.sum())

134.0
6416.0
0.0
4010.0
0.0
2410.0
100.0


In [ ]:
print(train_X0h1.shape)
for i in train_X0h1[20][50]:
  print(i)

(6416, 126, 40)
-612.3342752179142
111.6959331888494
10.253096761455323
-6.948020278838494
6.4637502146761525
4.635434315719341
-7.07616975744697
-1.90056853881371
3.2065844058609048
-2.138529156920269
-10.56705131670065
-7.227762311551126
-1.4676623180174486
-4.809637376688481
-6.644084075272458
-1.7979116251875027
-0.8177400434671149
-6.04618422961991
-1.2579016471879245
-8.402311861480634
-0.5718797297939697
-2.4732177675335563
0.9131301510101417
1.9141967824569412
0.46019265441521706
0.4361513510124355
-0.8651543776746824
1.8635701138912943
3.157150031574242
0.7465361588249468
4.410159204162532
-1.5310318473438498
-2.3266577693366033
3.1430651323695487
1.4694258225732961
-0.7062088622905469
-0.9701974410299848
0.5870610167564396
0.6437750865294342
-0.3021659855227233


In [ ]:
train_X0 = np.concatenate((train_X0y1[:15000], train_X0g[:10000], train_X0h, test_X))
train_Y0 = np.concatenate((train_Y0y1[:15000], train_Y0g[:10000], train_Y0h, test_Y))

test_X0 = np.concatenate((train_X0cs, train_X0h, train_X0hg, train_X0y1[15000:], train_X0g[10000:]))
test_Y0 = np.concatenate((train_Y0cs, train_Y0h, train_Y0hg, train_Y0y1[15000:], train_Y0g[10000:]))

print(train_X0.shape, train_Y0.shape)
print(train_Y0.sum())

print(test_X0.shape, test_Y0.shape)
print(test_Y0.sum())

(28745, 126, 40) (28745,)
2510.0
(26984, 126, 40) (26984,)
6554.0


In [ ]:
randomize = np.arange(len(train_Y0))
np.random.shuffle(randomize)
train_X_shuffle = train_X0[randomize]
train_Y_shuffle = train_Y0[randomize]

In [ ]:
import pickle

with open("total_train_x_new.pickle", "wb") as f:
  pickle.dump(train_X_shuffle, f)
print("Train set features done")
with open("total_train_y_new.pickle", "wb") as f:
  pickle.dump(train_Y_shuffle, f)
print("Train set labels done")

Train set features done
Train set labels done


# Training on the data



In [ ]:
os.chdir("Help")
os.listdir()

['help_trimmed_train_x.pickle',
 'help_trimmed_train_y.pickle',
 'help_trimmed_test_x.pickle',
 'help_trimmed_test_y.pickle',
 'google_mix_help_train_x.pickle',
 'google_mix_help_train_y.pickle',
 'crowdsourced_train_x.pickle',
 'crowdsourced_train_y.pickle',
 'new_mix_help_train_x.pickle',
 'new_mix_help_train_y.pickle',
 'total_train_x_new.pickle',
 'total_train_y_new.pickle']

In [13]:
with open('total_train_x_new.pickle', 'rb') as f:
  train_X0 = pickle.load(f)
print("Train set features loaded")
with open('total_train_y_new.pickle', 'rb') as f:
  train_Y0 = pickle.load(f)
print("Train set labels loaded")

with open('crowdsourced_train_x.pickle', 'rb') as f:
  train_X0cs = pickle.load(f)
print("Crowdsourced Train set features loaded")
with open('crowdsourced_train_y.pickle', 'rb') as f:
  train_Y0cs = pickle.load(f)
print("Crowdsourced Train set labels loaded")

with open('help_trimmed_test_x.pickle', 'rb') as f:
  train_X0t = pickle.load(f)
print("Test set features loaded")
with open('help_trimmed_test_y.pickle', 'rb') as f:
  train_Y0t = pickle.load(f)
print("Test set labels loaded")

Train set features loaded
Train set labels loaded
Crowdsourced Train set features loaded
Crowdsourced Train set labels loaded
Test set features loaded
Test set labels loaded


In [14]:
print(train_X0.shape, train_Y0.shape)
print(train_Y0.sum())

(58266, 126, 40) (58266,)
12836.0


In [15]:
# randomize = np.arange(len(train_Y0))
# np.random.shuffle(randomize)
# train_X_shuffle = train_X0[randomize]
# train_Y_shuffle = train_Y0[randomize]

In [16]:
# print(train_X_shuffle.shape)
# print(train_Y_shuffle.shape)

In [17]:
import sklearn
train_X_cv, test_X_cv, train_Y_cv, test_Y_cv = sklearn.model_selection.train_test_split(train_X_shuffle, train_Y_shuffle, test_size=0.1)
# train_X_cv, test_X_cv, train_Y_cv, test_Y_cv = sklearn.model_selection.train_test_split(train_X0, train_Y0, test_size=0.1)

In [18]:
test_X1 = np.concatenate((test_X0, test_X_cv))
test_Y1 = np.concatenate((test_Y0, test_Y_cv))

In [19]:
def generator(features, labels, batch_size):
    # Create empty arrays to contain batch of features and labels#
    batch_features = np.zeros((batch_size, 126, 40))
    batch_labels = np.zeros((batch_size,1))
    while True:
        for i in range(batch_size):
            index= random.choice(len(features),1)
            batch_features[i] = features[index]
            batch_labels[i] = labels[index]
        yield batch_features, batch_labels

In [20]:
test_Y_cv.shape

(2875,)

In [21]:
class_weight = {True: 4.,
                False: 1.}

In [22]:
# inputs = keras.Input(shape = (126,40))    
# x = Bidirectional(LSTM(units=128,return_sequences=True))(inputs)
# x = GlobalMaxPool1D()(x)
# outputs = Dense(1, activation='sigmoid')(x)

# model = keras.Model(inputs, outputs)
# model.summary()
# opt = keras.optimizers.Adam(learning_rate=0.001)
# model.compile(loss='binary_crossentropy',
#               optimizer=opt,
#               metrics=['acc', Precision()])

# model.fit_generator(generator(train_X0[10000:], train_Y0[10000:], 128), class_weight = class_weight, verbose=1, steps_per_epoch=320, epochs=50, validation_data=(train_X0[:10000], train_Y0[:10000]))

In [ ]:
logdir = "../logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

inputs = keras.Input(shape = (126,40))    
x = Bidirectional(LSTM(units=128,return_sequences=True))(inputs)
x = GlobalMaxPool1D()(x)
# x = Dense(64, activation='sigmoid')(x)
outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)
model.summary()
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['acc', Precision()])

training_history = model.fit(train_X_cv, train_Y_cv, epochs=10, validation_split=0.2, callbacks=[tensorboard_callback])
print("Average test loss: ", np.average(training_history.history['loss']))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 126, 40)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 126, 256)          173056    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 173,313
Trainable params: 173,313
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
645/647 [============================>.] - ETA: 0s - loss: 0.1706 - acc: 0.9346 - precision_1: 0.7656

In [ ]:
#Uncomment to save the retrained model
model.save('../Help_Models/HDF5/help2.h5')
print("SAVED")

In [ ]:
# def create_model(INPUT_SHAPE):
#   print ('Creating model...')
#   model = Sequential()
#   model.add(Bidirectional(LSTM(units=128, input_shape=INPUT_SHAPE, return_sequences=True)))
#   model.add(GlobalMaxPool1D())
#   model.add(Dense(units=1,activation='sigmoid'))
#   opt = keras.optimizers.Adam(learning_rate=0.001)	
#   model.compile(loss='binary_crossentropy',
# 	              optimizer=opt,
# 	              metrics=['acc', Precision()])
#   return model

# model = create_model(INPUT_SHAPE = (126,40))

# training_history = model.fit(train_X_cv, train_Y_cv, epochs=5, validation_split=0.2, callbacks=[tensorboard_callback])
# print("Average test loss: ", np.average(training_history.history['acc']))

# Evaluation

In [ ]:
modelPath = '../Help_Models/HDF5/help1.h5'
model = tf.keras.models.load_model(modelPath)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 126, 40)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 126, 256)          173056    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 173,313
Trainable params: 173,313
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir /content/drive/'My Drive'/logs/scalars

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

In [ ]:
predictions = model.predict(test_X0)
y_pred = (predictions>0.95)
matrix = metrics.confusion_matrix(test_Y0, y_pred)
print(matrix)

[[20421     9]
 [ 3657  2897]]


In [ ]:
c=0
for i in range(y_pred.shape[0]):
  if y_pred[i]==True and test_Y0[i]==False:
    c=c+1
print(c)

7
